In [1]:
import sys
import os

sys.path.append(os.path.abspath('..'))

import yaml
import optuna

from src.data.load import load_data
from src.data.prepare import prepare_data
from src.models.cv_iterator import leave_last_k
from src.data.features import feature_engineering
from src.data.utils import build_rank_input
from src.models.tuner import BayesianSearch
from src.models.evaluator import evaluation

In [2]:
# read config
with open('../config.yml', 'r') as file:
    config=yaml.load(file, Loader= yaml.SafeLoader)
del file

In [3]:
# load and prepare data
dfs = load_data(config=config['data_loader'])
dfs = prepare_data(dataframes=dfs)

In [4]:
# train-test split
df_train, df_test = leave_last_k(df=dfs['data'], config=config['optimization'])
df_train, df_valid = leave_last_k(df=df_train, config=config['optimization'])

In [5]:
user_item_features = feature_engineering(
    dataframes={'user': dfs['user'], 'item': dfs['item'], 'data': df_train}
    )

In [6]:
import pandas as pd
from src.models.candidate import candidate_generation

df_train_neg = candidate_generation(df_train, n=20, positive_sampling=False)
df_train = pd.concat([df_train.iloc[:,:3], df_train_neg], ignore_index=True)

del df_train_neg

df_train, df_valid = [
    build_rank_input(ratings=df.iloc[:,:3], features=user_item_features) for df in (df_train, df_valid)
    ]

In [7]:
# perform bayesian search
searcher = BayesianSearch(config['optimization']['hyper_params'], algorithm='XGBRanker')

def objective(trial) -> float:
    return searcher.fit(df_train, df_valid, trial)

study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=20)

[I 2025-04-22 20:33:05,966] A new study created in memory with name: no-name-8cc3aa7d-4385-4e5d-9a49-831bd06ad03e
[I 2025-04-22 20:33:08,613] Trial 0 finished with value: 0.943762258524755 and parameters: {'learning_rate': 0.19919452259324139, 'gamma': 0.24989044257933968, 'max_depth': 9, 'subsample': 0.9458610345233347, 'n_estimators': 169}. Best is trial 0 with value: 0.943762258524755.
[I 2025-04-22 20:33:11,418] Trial 1 finished with value: 0.9477705768414255 and parameters: {'learning_rate': 0.0912024803712384, 'gamma': 3.8535621135237146, 'max_depth': 3, 'subsample': 0.825040631166075, 'n_estimators': 284}. Best is trial 1 with value: 0.9477705768414255.
[I 2025-04-22 20:33:12,461] Trial 2 finished with value: 0.9454059538705104 and parameters: {'learning_rate': 0.13851670177699218, 'gamma': 0.15554303593300733, 'max_depth': 6, 'subsample': 0.6319260803729445, 'n_estimators': 55}. Best is trial 1 with value: 0.9477705768414255.
[I 2025-04-22 20:33:13,633] Trial 3 finished with va

In [8]:
print("Best trial:")
print(study.best_trial.params)

Best trial:
{'learning_rate': 0.10593557246046809, 'gamma': 2.860170838693225, 'max_depth': 4, 'subsample': 0.836027815527547, 'n_estimators': 246}


In [9]:
import pandas as pd

# use indicator to know the origin
df_train = dfs['data'].merge(
    df_test
    , on=['user_id', 'item_id', 'rating'], how='left'
    , indicator=True
    )
# keep only rows that are present in df1 but not in df2
df_train = df_train[df_train['_merge'] == 'left_only'].drop(columns=['_merge'])

user_item_features = feature_engineering(
    dataframes={'user': dfs['user'], 'item': dfs['item'], 'data': df_train}
    )

# add negative sampling
df_train_neg = candidate_generation(df_train, n=20, positive_sampling=False)
df_train_ = pd.concat([df_train, df_train_neg], ignore_index=True)

del df_train_neg

df_train_, df_test_ = [
    build_rank_input(ratings=df.iloc[:,:3], features=user_item_features) for df in (df_train_, df_test)
    ]

In [10]:
from sklearn.metrics import ndcg_score
from xgboost import XGBRanker
import numpy as np

clf = XGBRanker(**study.best_trial.params)
clf.fit(
    df_train_['X'], df_train_['y'].astype(int)
    , group=df_train_['group']
    , verbose=False
    )

preds = clf.predict(df_test_['X'])

print(evaluation(df_test_['y'], preds, df_test_['group']))

0.952881871545766


In [11]:
def calculate_metrics(top_items, data):
    # Coverage: proportion of unique items recommended
    recommended_items = top_items['item_id'].unique()
    total_items = data['item_id'].unique()
    coverage_score = len(recommended_items) / len(total_items)
    
    # Novelty: Based on item popularity (inverse)
    item_popularity = data.groupby('item_id').size()
    item_popularity = item_popularity[item_popularity > 0]
    item_popularity = 1 - item_popularity / item_popularity.sum()  # Normalize
    novelty_score = top_items['item_id'].map(item_popularity).mean()

    return coverage_score, novelty_score

# Calculate metrics for the recommendations
coverage_score, novelty_score = calculate_metrics(df_test.iloc[:, :2], df_train.iloc[:, :3])
# Print out the results
print(f'Coverage: {coverage_score}')
print(f'Novelty: {novelty_score}')

Coverage: 0.6463195691202872
Novelty: 0.9984584266355963


In [12]:
from src.models.baseline import baseline_model

result = baseline_model(dataframes=dfs, n=10)
group = result.groupby('user_id').size().to_list()

print(evaluation(result['rating'], result['est_rating'], group))
del group

0.9168812846621082


In [13]:
user_item_features = feature_engineering(dataframes={
    'user': dfs['user'], 'item': dfs['item'], 'data': dfs['data']
    })

df_baseline = build_rank_input(ratings=result.drop(columns=['est_rating']), features=user_item_features)

preds = clf.predict(df_baseline['X'])
print(evaluation(df_baseline['y'], preds, df_baseline['group']))

0.948952904515824
